In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
img_folder = "/home/fdahle/Desktop/agi_test/images"
msk_folder = "/home/fdahle/Desktop/agi_test/masks"

In [3]:
"""
import src.base.create_mask as cm
import src.load.load_image as li
import src.export.export_tiff as et

import os
for img in os.listdir(img_folder):
    
    img_path = os.path.join(img_folder, img)
    img_id = img.split(".")[0]
    print(img_path)
    
    # load image
    img = li.load_image(img_path)
    mask = cm.create_mask(img, use_database=True, image_id=img_id)
    et.export_tiff(mask, os.path.join(msk_folder, f"{img_id}_mask.tif"), overwrite=True)
"""

'\nimport src.base.create_mask as cm\nimport src.load.load_image as li\nimport src.export.export_tiff as et\n\nimport os\nfor img in os.listdir(img_folder):\n    \n    img_path = os.path.join(img_folder, img)\n    img_id = img.split(".")[0]\n    print(img_path)\n    \n    # load image\n    img = li.load_image(img_path)\n    mask = cm.create_mask(img, use_database=True, image_id=img_id)\n    et.export_tiff(mask, os.path.join(msk_folder, f"{img_id}_mask.tif"), overwrite=True)\n'

In [4]:
"""
import src.base.rotate_image as ri
import src.load.load_image as li
import src.export.export_tiff as et
import os

# rotate vertical images 
for img in os.listdir(img_folder):
    if "V" in img:
        print(img)
        img_path = os.path.join(img_folder, img)
        img_id = img.split(".")[0]
        img = li.load_image(img_path)
        img = ri.rotate_image(img, 180)
        et.export_tiff(img, img_path, overwrite=True)

for img in os.listdir(msk_folder):
    if "V" in img:
        print(img)
        img_path = os.path.join(msk_folder, img)
        img_id = img.split(".")[0]
        img = li.load_image(img_path)
        img = ri.rotate_image(img, 180)
        et.export_tiff(img, img_path, overwrite=True)
"""

'\nimport src.base.rotate_image as ri\nimport src.load.load_image as li\nimport src.export.export_tiff as et\nimport os\n\n# rotate vertical images \nfor img in os.listdir(img_folder):\n    if "V" in img:\n        print(img)\n        img_path = os.path.join(img_folder, img)\n        img_id = img.split(".")[0]\n        img = li.load_image(img_path)\n        img = ri.rotate_image(img, 180)\n        et.export_tiff(img, img_path, overwrite=True)\n\nfor img in os.listdir(msk_folder):\n    if "V" in img:\n        print(img)\n        img_path = os.path.join(msk_folder, img)\n        img_id = img.split(".")[0]\n        img = li.load_image(img_path)\n        img = ri.rotate_image(img, 180)\n        et.export_tiff(img, img_path, overwrite=True)\n'

In [5]:
import os
import Metashape
import src.load.load_image as li

# get list with image paths from folder
images = [f"{img_folder}/{img}" for img in os.listdir(img_folder)]

# only keep images with "V" in the name
#images = [img for img in images if "V" in img]

doc = Metashape.Document()

chunk = doc.addChunk()

# add the images to the chunk
chunk.addPhotos(images)

# add masks to the images
for camera in chunk.cameras:
    msk_path = os.path.join(msk_folder, f"{camera.label}_mask.tif")
    mask = li.load_image(msk_path)
    
    m_mask = Metashape.Image.fromstring(mask, 
                                        mask.shape[1], mask.shape[0],
                                        channels=' ', datatype='U8')
    mask_obj = Metashape.Mask()
    mask_obj.setImage(m_mask)
    camera.mask = mask_obj


AddPhotos


In [6]:
import src.base.find_tie_points as ftp
import src.load.load_image as li
import src.display.display_images as di
import numpy as np
tp_dict = {}
conf_dict = {}

tpd = ftp.TiePointDetector('lightglue', min_conf_value=0.7)

# find matches between the images
for camera1 in chunk.cameras:
    for camera2 in chunk.cameras:
        if camera1.label == camera2.label:
            continue
            
        if (camera2.label, camera1.label) in tp_dict:
            continue
                
        print(camera1.label, camera2.label)
        
        img1 = li.load_image(os.path.join(img_folder, f"{camera1.label}.tif"))
        img2 = li.load_image(os.path.join(img_folder, f"{camera2.label}.tif"))
        if len(img1.shape) == 3:
            img1 = img1[0,:,:]
        if len(img2.shape) == 3:
            img2 = img2[0,:,:]
        
        mask1 = li.load_image(os.path.join(msk_folder, f"{camera1.label}_mask.tif"))
        mask2 = li.load_image(os.path.join(msk_folder, f"{camera2.label}_mask.tif"))
        
        
        # find tie points
        tps, conf = tpd.find_tie_points(img1, img2, mask1=mask1, mask2=mask2)
        
        # store tie points and confidence
        tp_dict[(camera1.label, camera2.label)] = tps
        conf_dict[(camera1.label, camera2.label)] = conf
        print(tps.shape)
        


CA215232V0360 CA215232V0361
(3017, 4)
CA215232V0360 CA215232V0362
(371, 4)
CA215232V0360 CA215331L0391
(0, 4)
CA215232V0360 CA215331L0392
(0, 4)
CA215232V0360 CA215331L0393
(0, 4)
CA215232V0361 CA215232V0362
(7187, 4)
CA215232V0361 CA215331L0391
(4, 4)
CA215232V0361 CA215331L0392
(141, 4)
CA215232V0361 CA215331L0393
(7, 4)
CA215232V0362 CA215331L0391
(2, 4)
CA215232V0362 CA215331L0392
(44, 4)
CA215232V0362 CA215331L0393
(382, 4)
CA215331L0391 CA215331L0392
(14011, 4)
CA215331L0391 CA215331L0393
(7220, 4)
CA215331L0392 CA215331L0393
(11035, 4)


In [9]:
import src.display.display_images as di
for key, val in tp_dict.items():
    
    if val.shape[0] == 0:
        continue
    
    style_config = {
        'title': f"{key[0]} - {key[1]}: {val.shape[0]}",
    }
    img1 = li.load_image(key[0])
    img2 = li.load_image(key[1])
    
    di.display_images([img1, img2], tie_points=val, style_config=style_config)
    

In [ ]:
def find_camera_by_label(chunk, label):
    for camera in chunk.cameras:
        if camera.label == label:
            return camera
    return None


In [ ]:
doc.save("/home/fdahle/Desktop/agi_test/agi_test.psx")

In [ ]:
import os
from plyfile import PlyData

fld = "/home/fdahle/Desktop/agi_test/point_cloud_small"

# iterate all ply files in a fld
for file in os.listdir(fld):
    
    if not file.endswith(".ply"):
        continue
    
    file_path = os.path.join(fld, file)
    output_path = file_path.replace(".ply", ".csv")
    
    ply_data = PlyData.read(file_path)
    vertex_data = ply_data['vertex'].data
    column_names = vertex_data.dtype.names  # Get the names of all columns
    vertex_arr= np.vstack([vertex_data[name] for name in column_names]).T
    
    print(file, vertex_arr.shape)
    np.savetxt(output_path, vertex_arr, delimiter=";")


In [17]:
points_per_image = {}

print(len(tp_dict))

# iterate over all tie points
for key, value in tp_dict.items():
    
    # get the two ids
    img1_id = key[0]
    img2_id = key[1]
    
    if value.shape[0] < 25:
        continue
    
    # create an empty numpy array for each image if it does not exist
    if img1_id not in points_per_image:
        points_per_image[img1_id] = np.zeros((0, 2))
    if img2_id not in points_per_image:
        points_per_image[img2_id] = np.zeros((0, 2))
    
    # get the points per image
    points1 = value[:, :2]
    points2 = value[:, 2:]

    # add the points to the numpy array
    points_per_image[img1_id] = np.vstack((points_per_image[img1_id], points1))
    points_per_image[img2_id] = np.vstack((points_per_image[img2_id], points2))
    
import pandas as pd
import src.export.export_ply as ep
counter = 0
for key, val in points_per_image.items():
            
    # remove duplicates
    val = np.unique(val, axis=0)
    
    # skip tie-points with less than 25 points
    if val.shape[0] < 25:
        continue
    
    # convert numpy to pandas
    df = pd.DataFrame(val, columns=["x", "y"])
    
    # add column size with default value of 10
    df["size"] = 10.0
    
    # add column id with a range of 0 to the number of points
    df["id"] = range(len(df))
    
    # export as ply
    ply_path = f"/home/fdahle/Desktop/agi_test/custom_vertical/p{counter}.ply"
    ep.export_ply(df, ply_path)
    
    print(df.shape)

    counter = counter + 1
    

15
(3068, 4)
(10333, 4)
(7973, 4)
(24841, 4)
(18507, 4)
(15553, 4)


In [19]:
import os
import struct

import xml.etree.ElementTree as ET
folder = "/home/fdahle/Desktop/agi_test/point_cloud_vertical"

xml_path = os.path.join(folder, "doc.xml")
# Load the XML file
tree = ET.parse(xml_path)
root = tree.getroot()


def save_ply_file(file_path, points):
    """
    Saves a PLY file with given points.
    :param file_name: Name of the output PLY file.
    :param points: List of points where each point is (x, y, z, id).
    """
    with open(file_path, 'wb') as file:
        # PLY header
        header = (
            "ply\n"
            "format binary_little_endian 1.0\n"
            f"element vertex {len(points)}\n"
            "property float x\n"
            "property float y\n"
            "property float z\n"
            "property int id\n"
            "end_header\n"
        )
        file.write(header.encode('ascii'))
        
        # Write point data
        for point in points:
            file.write(struct.pack('<fffI', point[0], point[1], point[2], point[3]))
    
    # get file name
    file_name = os.path.basename(file_path)
    
    # Find the correct element in the XML and update the count attribute
    for elem in root.findall('projections'):
        if elem.attrib['path'] == file_name:
            elem.set('count', str(len(points)))


def generate_ply_files(tie_points_dict, fld):
    """
    Generates PLY files from the tie points dictionary.
    :param tie_points_dict: Dictionary where the key is (image1, image2) and the value is a numpy array with shape (x,4).
    """
    image_names = sorted(set([key[0] for key in tie_points_dict.keys()] + [key[1] for key in tie_points_dict.keys()]))
    image_to_index = {name: idx for idx, name in enumerate(image_names)}

    # Initialize a dictionary to hold the points for each image
    image_points = {image: [] for image in image_names}
    point_id = 0

    # Iterate over the tie points
    for (image1, image2), points in tie_points_dict.items():
        for point in points:
            x1, y1, x2, y2 = point
            
            # Add point to image1's PLY data
            image_points[image1].append((x1, y1, 10.0, point_id))
            
            # Add point to image2's PLY data
            image_points[image2].append((x2, y2, 10.0, point_id))
            
            # Increment the ID for the next point
            point_id += 1

    # Save PLY files for each image
    for image, points in image_points.items():
        index = image_to_index[image]
        file_name = f"p{index}.ply"
        file_path = os.path.join(fld, file_name)
        save_ply_file(file_path, points)


def save_tracks_ply_file(file_path, num_matches):
    """
    Saves a PLY file with the lengths of all matches.
    :param file_path: Path of the output PLY file.
    :param matches: List of match lengths.
    """
        
    with open(file_path, 'wb') as file:
        # PLY header
        header = (
            "ply\n"
            "format binary_little_endian 1.0\n"
            f"element vertex {num_matches}\n"
            "property uchar color\n"
            "end_header\n"
        )
        file.write(header.encode('ascii'))
        
        # Write match lengths with ID 1
        for match_length in range(num_matches):
            file.write(struct.pack('<B', 1))  # Using 1 as the color/ID (uchar)


# remove all non vertical images
tp_vertical = {}
for key, val in tp_dict.copy().items():
    if "V" in key[0] and "V" in key[1]:
        tp_vertical[key] = val

# get number of matches
num_matches = 0
for key, val in tp_vertical.items():
    num_matches += val.shape[0]

print(num_matches)

generate_ply_files(tp_vertical, folder)
save_tracks_ply_file(os.path.join(folder, "tracks.ply"), num_matches)
# Save the updated XML back to file
tree.write(xml_path)


10575


In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

xml_path = os.path.join(folder, "doc.xml")

# Load the XML file
tree = ET.parse(xml_path)
root = tree.getroot()

# Define the file paths
csv_files = {
    "p0": '/mnt/data/p0.csv',
    "p1": '/mnt/data/p1.csv',
    "p2": '/mnt/data/p2.csv',
}

# Update the counts in the XML
for camera_id, file_path in csv_files.items():
    # Load the CSV file
    data = pd.read_csv(file_path, delimiter=';')
    
    # Count the number of unique IDs
    unique_ids = data.iloc[:, -1].nunique()
    
    # Find the correct element in the XML and update the count attribute
    for elem in root.findall('projections'):
        if elem.attrib['camera_id'] == camera_id[-1]:  # camera_id corresponds to last character of key
            elem.set('count', str(unique_ids))

# Also update the tracks count with the number of unique IDs in tracks.csv
tracks_data = pd.read_csv('/mnt/data/tracks.csv', delimiter=';')
unique_tracks = tracks_data.iloc[:, 0].nunique()

for elem in root.findall('tracks'):
    elem.set('count', str(unique_tracks))

# Save the updated XML back to file
tree.write('/mnt/data/updated_doc.xml')


In [ ]:
# load tracks.csv
import pandas as pd
import os
folder = "/home/fdahle/Desktop/agi_test/point_cloud"
tracks = pd.read_csv(os.path.join(folder, "tracks.csv"), delimiter=";", header=None, names=["point_id"])

# count unique values
unique_values = tracks["point_id"].value_counts()
print(unique_values)
print(len(unique_values))

In [ ]:
import os
import zipfile

def create_zip_from_folder(folder_path, output_zip_path):
    """
    Creates a zip file containing all .ply and .xml files in a specified folder.
    :param folder_path: Path to the folder containing the files.
    :param output_zip_path: Path where the output zip file will be saved.
    """
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_STORED) as zipf:
        for foldername, subfolders, filenames in os.walk(folder_path):
            for filename in filenames:
                if filename.endswith('.ply') or filename.endswith('.xml'):
                    file_path = os.path.join(foldername, filename)
                    zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Example usage
folder_path = "/home/fdahle/Desktop/agi_test/point_cloud_new"
output_zip_path = "/home/fdahle/Desktop/agi_test/point_cloud_new.zip"
create_zip_from_folder(folder_path, output_zip_path)


In [ ]:
import Metashape
doc = Metashape.Document()
doc.open("/home/fdahle/Desktop/agi_test/agi_small.psx")
chunk = doc.chunk
chunk.alignCameras(adaptive_fitting=True, reset_alignment=True)
doc.save()